# Function dissection lab

1. What happens when we define a function?
2. Byte codes and Python compilation
3. Some attributes of the `__code__` attribute of a function object

In [1]:
def hello(name):
    return f'Hello, {name}!'

# What is the above?

I defined a function.

Let's be more specific:

1. I created a function object.
2. I assigned that function object to a variable -- in this case, `hello`.

In Python, everything is an object. That means: Everything follows the same rules.   This means that while we often think of functions as verbs, which do things, they are also nouns -- which we can store in variables, pass to other functions as arguments, and everything else you can do with strings, lists, tuples, dicts, etc.

When we define our function, Python's compiler looks at the function's definition, and writes all sorts of hints to itself for how to execute things down the line.

# Wait -- Python is compiled?

Python is compiled in the same way that Java and .NET are compiled:

1. First, the code is turned into a universal assembly language -- in our case, they're known as bytecodes.  This happens once, when the function is defined.
2. Then, those bytecodes are executed by a Python interpreter every time we run the funciton.

In the Python world, it's extremely rare to separate the compilation and runtime steps. Normally, when you "run" a Python program, it's first compiled and then executed.

In [2]:
# our function, hello, was compiled.
# let's take a look at the function object

type(hello)  # remember -- hello is a variable referring to a function!

function

In [3]:
# since hello refers to a function object
# since all objects in Python have attributes
# it stands to reason that our function object has attributes
# it does -- including the __code__ attribute, which contains
#  the byte-compiled core of the function object

hello.__code__   # "dunder code" -- "double underscore, before and after, code"

<code object hello at 0x10d7da290, file "/var/folders/d9/v8tsklln4477fll05wkgcpth0000gn/T/ipykernel_18597/3631425946.py", line 1>

In [4]:
# what's truly interesting is that this __code__ object also has attributes
dir(hello.__code__)  # what attributes does this object have?

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'co_argcount',
 'co_cellvars',
 'co_code',
 'co_consts',
 'co_filename',
 'co_firstlineno',
 'co_flags',
 'co_freevars',
 'co_kwonlyargcount',
 'co_lines',
 'co_linetable',
 'co_lnotab',
 'co_name',
 'co_names',
 'co_nlocals',
 'co_posonlyargcount',
 'co_stacksize',
 'co_varnames',
 'replace']

In [5]:
# let's look at the bytecodes themselves!
hello.__code__.co_code

b'd\x01|\x00\x9b\x00d\x02\x9d\x03S\x00'

In [6]:
# how can we turn the bytecodes into something readable, and understand
# what Python is doing?

# we can use the dis.dis function, which comes in Python's standard library
import dis
dis.dis(hello)

  2           0 LOAD_CONST               1 ('Hello, ')
              2 LOAD_FAST                0 (name)
              4 FORMAT_VALUE             0
              6 LOAD_CONST               2 ('!')
              8 BUILD_STRING             3
             10 RETURN_VALUE


In [7]:
# does Python have constants? NO
# do Python functions have constants? YES

# they are stored on the function object, based on the compiler's reading
# of our function body

# these are all of the literal values - -integers, strings, etc. -- that
# can be stored once, and then used multiple times. (Even if we're not going
# to use them multiple times, the compiler does this)

In [8]:
hello.__code__.co_consts  # let's see the constants!

(None, 'Hello, ', '!')

In [9]:
# parameters: name
# arguments: 'world'

# Python assigns 'world' to name (positional argument, assigned based on position)

hello('world')  

'Hello, world!'

In [10]:
# what happens if I call hello with zero arguments?

hello()

TypeError: hello() missing 1 required positional argument: 'name'

In [11]:
# how did Python know that we needed to pass 1 positional argument?
# how did Python know that the parameter to which it would be assigned is
#   called "name"?

# let's look in our function object to find out!

hello.__code__.co_argcount  # how many arguments does the function expect?

1

In [12]:
hello.__code__.co_varnames  # what are the names of the local variables?

('name',)

In [13]:
# since there's only one local variable
# and since argcount is 1,
# that local variable is a parameter -- it must get a value from the caller

In [14]:
# what happens if I call the function with too many arguments?
hello('out', 'there')

TypeError: hello() takes 1 positional argument but 2 were given

In [16]:
# wait -- how about when we have a default argument for a parameter?

def hello(name='(no name)'):   # name has a default value
    return f'Hello, {name}!'

In [17]:
# how does Python know when to use this default?
# where is the default stored?

In [18]:
hello.__defaults__  # this is a tuple of values, with default values

('(no name)',)

In [19]:
# If I call hello with 1 argument:

# parameters: name
# arguments: 'world'

# 'world' --> name

hello('world')

'Hello, world!'

In [20]:
# parameters: name
# arguments: '(no name)', taken from __defaults__

hello()

'Hello, (no name)!'

In [21]:
# can I have a function with more than one default? YES!
# parameters with defaults *MUST* come after parameters without defaults

# meaning: mandatory parameters before optional parameters

In [22]:
def add(first, second):
    return first + second

add(10, 3)

13

In [23]:
# let's add some defaults

def add(first=3, second=8):
    return first + second

In [24]:
# parameters: first second
# arguments:    3     8

add()   # nothing -- both values are taken from __defaults__

11

In [25]:
add.__defaults__

(3, 8)

In [26]:
# what if I pass a value for first, but not second?

# parameters: first  second
# arguments:   4       8

add(4)

12

In [27]:
# what if I pass a value for second, but not first?

add(second=9)  # using keyword arguments, in name=value form, will work

12

# Always:

1. When defining a function, mandatory parameters (without defaults) must come before optional parameters (with defaults)
2. When calling a function, positional arguments need to come before keyword arguments (i.e., those in the form of `name=value`).

# Dangers with defaults

In [28]:
def add_one(x):
    x.append(1)
    return x
    
mylist = [10, 20, 30]      # define a list
add_one(mylist)            # should return [10, 20, 30, 1]

[10, 20, 30, 1]

In [29]:
mylist   # it was changed by the function call!

[10, 20, 30, 1]

In [30]:
# if I run it again, we'll add another 1

add_one(mylist)
mylist

[10, 20, 30, 1, 1]

In [32]:
x = 7
x = 5

print(x)  # of course it'll print 5 -- that's the most recent value assigned to x!

5


In [33]:
# let's change add_one a bit

def add_one(x=[]):   # now we have a default value for x, []
    x.append(1)
    return x
    
add_one()     # what will be returned?

[1]

In [34]:
add_one()    # what will be returned this second time?

[1, 1]

In [35]:
# let's look through this again

def add_one(x=[]):   # now we have a default value for x, []
    x.append(1)
    return x

In [36]:
add_one.__defaults__   # what default value is there?

([],)

Defaults, and the values in the `__defaults__` tuple, are defined at **compile time**, not at run time.  Which means that if we have a mutable value, such as a list, in `__defaults__`, and then we modify it inside of our function body, that modified value will then be used as a default on the next run.

**THIS IS VERY BAD!**

The solution: Never, ever, ever use mutable defaults.

(Most IDEs and code checkers will point to this, and tell you not to.)

In [37]:
# How could/should I rewrite add_one to do the right thing, without mutable defaults?

def add_one(x=None):    # x is None, thus immutable

    # at runtime, I can potential reassign x
    if x is None:
        x = []    # this list is created at runtime, and thus is new with each invocation
    
    x.append(1)
    return x

In [38]:
add_one()

[1]

In [39]:
add_one()

[1]

In [40]:
add_one()

[1]

# Scoping

Python has very consistent, clear scoping rules. ("Scoping" refers to when variables are available, defined, and also when they disappear. If you talk about local vs. global variables, that's scoping.)

In [41]:
# for starters

x = 100

for i in range(5):
    x = i**2  # is this x the same as the x I defined on line 3?  Or is it local to the loop?
    
print(x)       # what will x's value be here?

16


# Scoping rules in Python

There are only four scopes in Python:

- `L` - Local -- we start searching here if we're inside of a function body
- `E` - (Enclosing)
- `G` - Global -- we start searching here if we're *not* inside of a function body
- `B` - Builtin

Python starts at either `L` or `G`, and keeps searching down the list, in order, until it either finds the variable it's looking for at that level or it gets to the end, and raises a `NameError`.

In [42]:
# we're not in a function body, so Python starts searching at the global level
# it asks: is x global?  We can check by looking for the key 'x' in globals()

x = 100

print(f'x = {x}')

x = 100


In [43]:
'x' in globals()

True

In [44]:
globals()['x']  # retrieve the global

100

In [45]:
x = 100

def myfunc():
    print(f'In myfunc, x = {x}')  # is x local? No. is x global? Yes!
    
print(f'Before, x = {x}')  # is x global? YES.
myfunc()
print(f'After, x = {x}')   # is x global? yes.

Before, x = 100
In myfunc, x = 100
After, x = 100


In [46]:
# how does Python know if x is local?
# it can check on the __code__.co_varnames tuple

'x' in myfunc.__code__.co_varnames

False

In [49]:
x = 100   # this is the global variable x!

def myfunc():
    x = 200     # this is the local variable x! No relation to global x
    print(f'In myfunc, x = {x}')    # is x local? YES!  The value is 200
    
# now what will be printed?

print(f'Before, x = {x}')  # is x global? yes, 100 
myfunc()
print(f'After, x = {x}')   # is x global? yes, 100

Before, x = 100
In myfunc, x = 200
After, x = 100


In [50]:
# is x local to myfunc?

myfunc.__code__.co_varnames

('x',)

In [51]:
# what happens if I do this:

x = 100  

def myfunc():
    # is x local? YES -- assigning to a variable inside of a function makes it local

    print(f'In myfunc, x = {x}')     # swap these two lines from before
    x = 200 
    
# now what will be printed?

print(f'Before, x = {x}')  # is x global? yes, 100
myfunc()
print(f'After, x = {x}')   

Before, x = 100


UnboundLocalError: local variable 'x' referenced before assignment

In [52]:
# what are the bytecodes for myfunc?

dis.dis(myfunc)

  8           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('In myfunc, x = ')
              4 LOAD_FAST                0 (x)
              6 FORMAT_VALUE             0
              8 BUILD_STRING             2
             10 CALL_FUNCTION            1
             12 POP_TOP

  9          14 LOAD_CONST               2 (200)
             16 STORE_FAST               0 (x)
             18 LOAD_CONST               0 (None)
             20 RETURN_VALUE


In [53]:
x = 100  

def myfunc():
    global x   # this tells Python's compiler *not* to tag x as local, but rather as global
    x = 200    # now, assignments to x go to the global x
    print(f'In myfunc, x = {x}')  
    
print(f'Before, x = {x}')  
myfunc()
print(f'After, x = {x}')   

Before, x = 100
In myfunc, x = 200
After, x = 200


In [54]:
dis.dis(myfunc)

  5           0 LOAD_CONST               1 (200)
              2 STORE_GLOBAL             0 (x)

  6           4 LOAD_GLOBAL              1 (print)
              6 LOAD_CONST               2 ('In myfunc, x = ')
              8 LOAD_GLOBAL              0 (x)
             10 FORMAT_VALUE             0
             12 BUILD_STRING             2
             14 CALL_FUNCTION            1
             16 POP_TOP
             18 LOAD_CONST               0 (None)
             20 RETURN_VALUE


In [55]:
def myfunc():
     x = 200   # this x is only local, only exists when we call myfunc, and inside of the function body
        
print(x)       # this will print a global value x, if it has been defined. If not, error!
x = 100        # here, we define x as a global variable, with a value 100

200


In [56]:
# show_code doesn't show byte codes. It shows us all of the hints
# that Python wrote down for our function

dis.show_code(hello)

Name:              hello
Filename:          /var/folders/d9/v8tsklln4477fll05wkgcpth0000gn/T/ipykernel_18597/3954879175.py
Argument count:    1
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  1
Stack size:        3
Flags:             OPTIMIZED, NEWLOCALS, NOFREE
Constants:
   0: None
   1: 'Hello, '
   2: '!'
Variable names:
   0: name


In [57]:
# the final place that Python looks for things is "builtins"
# these are very familiar to use -- print, len, int, str, list, dict, etc.
# which are not keywords !  

In [58]:
dir(__builtins__)

['ArithmeticError',
 'AssertionError',
 'AttributeError',
 'BaseException',
 'BlockingIOError',
 'BrokenPipeError',
 'BufferError',
 'BytesWarning',
 'ChildProcessError',
 'ConnectionAbortedError',
 'ConnectionError',
 'ConnectionRefusedError',
 'ConnectionResetError',
 'DeprecationWarning',
 'EOFError',
 'Ellipsis',
 'EncodingWarning',
 'EnvironmentError',
 'Exception',
 'False',
 'FileExistsError',
 'FileNotFoundError',
 'FloatingPointError',
 'FutureWarning',
 'GeneratorExit',
 'IOError',
 'ImportError',
 'ImportWarning',
 'IndentationError',
 'IndexError',
 'InterruptedError',
 'IsADirectoryError',
 'KeyError',
 'KeyboardInterrupt',
 'LookupError',
 'MemoryError',
 'ModuleNotFoundError',
 'NameError',
 'None',
 'NotADirectoryError',
 'NotImplemented',
 'NotImplementedError',
 'OSError',
 'OverflowError',
 'PendingDeprecationWarning',
 'PermissionError',
 'ProcessLookupError',
 'RecursionError',
 'ReferenceError',
 'ResourceWarning',
 'RuntimeError',
 'RuntimeWarning',
 'StopAsyncIt

In [ ]:
globals()   # what global values are defined

# Best practices (my favorite ones)

1. Optimize for readability, not execution speed.
2. Idiomatic Python is usually the best -- fastest to execute, easiest to read.
3. Write classes when you need to, not for everything.
4. Use the standard library! Or PyPI, if your solution isn't there.

# Book about Python internals

Anthony Shaw's CPython Internals